# Appendix: Project Code

- Project: Vehicle License Plate Detection using YOLOv8
- Group: 07
- ID: N01555914
- Name: Wenhao Fang

---


## Device Setup

- Enable Cuda

In [ ]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


- Mount the drive and set the path

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/ML_Ass10

/content/drive/MyDrive/ML_Ass10


## Data Collection

- `simple_image_download` :

  - a Python library that allows to search URLs of images from “Google images” using tags, and downloads them to local computer.
  - In this Assignment, Version 0.4 will be used for better compatiable.

- Command to install `simple_image_download`:

```sh
pip install simple-image-download==0.4
```


In [ ]:
# pip install simple-image-download==0.4
# Define downloady_images function
from simple_image_download import simple_image_download as simp

def download_images(keyword, num_pic):
    response = simp.simple_image_download()

    for kw in keyword:
        response.download(keywords=kw, limit=num_pic)

In [ ]:
# Define a list of keywords to download images
keywords = [
    "Corvette with license plate",
    "Volkswagen Beetle with license plate",
    "Volkswagen Golf with license plate",
    "Volkswagen Passat with license plate",
    "Volkswagen Tiguan with license plate",
    "Toyota Tacoma with license plate",
    "Toyota Corolla with license plate",
    "Toyota Highlander with license plate",
    "Toyota Camry with license plate",
    "Toyota RAV4 with license plate",
    "Honda Accord with license plate",
    "Honda Pilot with license plate",
    "Honda CR-V with license plate",
    "Honda Odyssey with license plate",
    "Honda Passport with license plate",
    "Honda Civic with license plate",
    "Honda HR-V with license plate",
    "BMW with license plate",
    "Audi Q8 with license plate",
    "Audi A3 Convertible with license plate",
    "Audi A5 with license plate",
    "Audi A4 with license plate",
    "Audi Q7 with license plate",
    "Audi A8 with license plate",
    "Hyundai Kona with license plate",
    "Hyundai Sonata with license plate",
    "Hyundai Elantra with license plate",
    "Hyundai Santa Cruz with license plate",
    "Hyundai Accent with license plate",
    "Jeep Gladiator with license plate",
    "Jeep Grand Cherokee with license plate",
    "Jeep Avenger with license plate",
    "Chevrolet Camaro with license plate",
    "Chevrolet Corvette with license plate",
    "Chevrolet Malibu with license plate",
    "Ford F-Series with license plate",
    "Ford Escort with license plate",
    "Ford Fiesta with license plate",
    "Ford Focus with license plate",
    "Ford Mustang with license plate",
    "Ford Transit with license plate",
    "Ford Taurus with license plate",
    "Ford Explorer with license plate",
    "Tesla Model Y with license plate",
    "tesla model 3 with license plate",
    "tesla model s with license plate",
    "car with license plate",
    "vehicle with license plate",
    "sports car with plate",
]
# Define max number of pic to download
num_pic = 200

# Call function to dowload
# download_images(keywords, num_pic)  # comment after download

[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[=======================================

---


## Data Processing

### Image Annotation

- `LabelImg`

  - a **graphical image annotation tool**.
  - It is written in Python and uses Qt for its **graphical interface**.
  - Annotations are saved as `XML` files in `PASCAL VOC` format, the format used by `ImageNet`. Besides, it also supports `YOLO` and `CreateML` formats.

  - Github: https://github.com/HumanSignal/labelImg

- Command to install and call programme

```sh
# Install labelImg library
pip install labelImg

# call programme
labelImg
```


---

## Configure YOLOv8 Environment and Model

### Dataset Architecture

- Create directory for train, val, and pred datasets.
- Split images into train, val, and pred datasets.
- Copy images and annotation files to train, val, and pred directories.


In [ ]:
# Define functions for data processing
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split

# region Supporting Functions


def create_datasets_dir(dir):
    '''Create train and test directories'''

    if os.path.isdir(dir):
        shutil.rmtree(dir)

    Path(dir).mkdir(parents=True, exist_ok=True)

    train_path = Path(dir, "train")
    train_image_path = Path(train_path, "images")
    train_labels_path = Path(train_path, "labels")

    Path(train_path).mkdir(parents=True, exist_ok=True)
    Path(train_image_path).mkdir(parents=True, exist_ok=True)
    Path(train_labels_path).mkdir(parents=True, exist_ok=True)

    pred_path = Path(dir, "pred")
    pred_image_path = Path(pred_path, "images")
    pred_labels_path = Path(pred_path, "labels")

    Path(pred_path).mkdir(parents=True, exist_ok=True)
    Path(pred_image_path).mkdir(parents=True, exist_ok=True)
    Path(pred_labels_path).mkdir(parents=True, exist_ok=True)

    val_path = Path(dir, "val")
    val_image_path = Path(val_path, "images")
    val_labels_path = Path(val_path, "labels")

    Path(val_path).mkdir(parents=True, exist_ok=True)
    Path(val_image_path).mkdir(parents=True, exist_ok=True)
    Path(val_labels_path).mkdir(parents=True, exist_ok=True)


def copy_files(source_fn_list, source_path,  target_path):
    '''Copy selected file list to destination'''

    image_path = Path(target_path, "images")
    labels_path = Path(target_path, "labels")
    for fn in source_fn_list:
        # if the csv file exists
        if Path(source_path, fn+".txt").is_file() and Path(source_path, fn+".jpeg").is_file():
            shutil.copy(Path(source_path, fn+".jpeg"), image_path)
            shutil.copy(Path(source_path, fn+".txt"), labels_path)


def report_info(target_path):
    '''report files in path'''
    report = {
        "images": {"path": Path(target_path, "images"), "count": 0},
        "labels": {"path": Path(target_path, "labels"), "count": 0},
    }

    for file in os.listdir(Path(target_path, "labels")):
        if file.endswith(".txt"):
            report['labels']["count"] += 1

    for file in os.listdir(Path(target_path, "images")):
        if file.endswith(".jpeg"):
            report["images"]["count"] += 1

    return report


def clear_runr_dir(dir):
    # Clean left over runs dir
    if os.path.isdir(dir):
        shutil.rmtree("./runs")

# endregion

In [ ]:
# Define paths
from pathlib import Path

ROOT_DIR = Path().resolve() # Root path

LABEL_IMAGE_PATH = Path(ROOT_DIR, "data_label_image")
DATASET_PATH = Path(ROOT_DIR, "datasets")
TRAIN_DS_PATH = Path(DATASET_PATH, "train")
PRED_DS_PATH = Path(DATASET_PATH, "pred")
VAL_DS_PATH = Path(DATASET_PATH, "val")

# train/val/pred: 80:10:10
VAL_SIZE = 0.10 # % of val size
PRED_SIZE = 0.10 # % of pred size

# establish datasets directory
create_datasets_dir(DATASET_PATH)

# get unique list of file
fn_list = os.listdir(LABEL_IMAGE_PATH)
fn_unique_list = list(set(["".join(fn.split(".")[:-1]) for fn in fn_list]))

# split train, val, and pred dataset
fn_train, fn_val_pred = train_test_split(
    fn_unique_list, test_size=VAL_SIZE+PRED_SIZE, random_state=42)    # get train set, with test_size = val+pred

fn_val, fn_pred = train_test_split(
    fn_val_pred, test_size=VAL_SIZE/(PRED_SIZE+VAL_SIZE), random_state=42)    # get val and pred set, with test_size = val/val+pred

# # clone files to destination
copy_files(fn_train, LABEL_IMAGE_PATH,  TRAIN_DS_PATH)
copy_files(fn_val, LABEL_IMAGE_PATH,  VAL_DS_PATH)
copy_files(fn_pred, LABEL_IMAGE_PATH,  PRED_DS_PATH)

# train_report
train_report = report_info(TRAIN_DS_PATH)
print(
    f"\nReport: Train Dataset\nImages Path: {train_report['images']['path']}\nImages Count: {train_report['images']['count']}\nLabel Path: {train_report['labels']['path']}\nLabels Count: {train_report['labels']['count']}")

# val_report
val_report = report_info(VAL_DS_PATH)
print(
    f"\nReport: Val Dataset\nImages Path: {val_report['images']['path']}\nImages Count: {val_report['images']['count']}\nLabel Path: {val_report['labels']['path']}\nLabels Count: {val_report['labels']['count']}")

# pred_report
pred_report = report_info(PRED_DS_PATH)
print(
    f"\nReport: Pred Dataset\nImages Path: {pred_report['images']['path']}\nImages Count: {pred_report['images']['count']}\nLabel Path: {pred_report['labels']['path']}\nLabels Count: {pred_report['labels']['count']}")

clear_runr_dir("./runs")



Report: Train Dataset
Images Path: /content/drive/MyDrive/ML_Ass10/datasets/train/images
Images Count: 431
Label Path: /content/drive/MyDrive/ML_Ass10/datasets/train/labels
Labels Count: 431

Report: Val Dataset
Images Path: /content/drive/MyDrive/ML_Ass10/datasets/val/images
Images Count: 56
Label Path: /content/drive/MyDrive/ML_Ass10/datasets/val/labels
Labels Count: 56

Report: Pred Dataset
Images Path: /content/drive/MyDrive/ML_Ass10/datasets/pred/images
Images Count: 56
Label Path: /content/drive/MyDrive/ML_Ass10/datasets/pred/labels
Labels Count: 56


---

### YOLO Model Configuration

- YAML file in `datasets` directory

```yaml
names:
  0: <label_name>
train: <absolute path of train dataset's images>
val: <absolute path of val dataset's images>
```


In [ ]:
import yaml

YAML_PATH = "./datasets/config.yaml"  # path to data file

data = dict(
    train=Path(TRAIN_DS_PATH, "images").absolute().as_posix(),
    val=Path(VAL_DS_PATH, "images").absolute().as_posix(),
    names={0: "car_plate"}
)

with open(YAML_PATH, 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

## Train a custom YOLOv8 Detector

- Train custom model from scratch.


In [ ]:
# !pip install ultralytics
from ultralytics import YOLO

YOLO_V8N = "./model/base/yolov8n.pt"  # the model to train customized model

# Define model object from default v8n model
car_plate_model = YOLO(YOLO_V8N)
car_plate_model.train(data=YAML_PATH)    # Train Yolo model, using mps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.9/645.9 kB 7.9 MB/s eta 0:00:00
Ultralytics YOLOv8.0.219 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=./model/base/yolov8n.pt, data=./datasets/config.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

100%|██████████| 755k/755k [00:00<00:00, 24.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 80.6MB/s]


AMP: checks passed ✅


...

                   all         56         58      0.978          1      0.994      0.878



100 epochs completed in 0.646 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.219 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.82it/s]


                   all         56         58      0.998      0.966      0.994      0.889
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7895a3fe0880>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

---

## Predict using Custom YOLO Model


In [8]:
# ! pip install ultralytics

from ultralytics import YOLO
from pathlib import Path

CAR_PLATE_MODEL_PATH = "./runs/detect/train/weights/best.pt"
PRED_IMAGE_PATH = Path(".", "datasets", "pred", "images").absolute().as_posix()

# Load a model
car_plate_model = YOLO(CAR_PLATE_MODEL_PATH)  # pretrained YOLOv8n model

car_plate_model.predict(PRED_IMAGE_PATH, save=True, conf=0.5)


image 1/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/Audi A3 Convertible with license plate_57.jpeg: 448x640 1 car_plate, 423.6ms
image 2/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/Audi A3 Convertible with license plate_6.jpeg: 480x640 1 car_plate, 293.2ms
image 3/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/Audi A4 with license plate_68.jpeg: 448x640 1 car_plate, 201.0ms
image 4/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/Audi A5 with license plate_38.jpeg: 384x640 1 car_plate, 151.4ms
image 5/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/Audi Q7 with license plate_16.jpeg: 480x640 1 car_plate, 181.7ms
image 6/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/Audi Q7 with license plate_50.jpeg: 480x640 1 car_plate, 178.0ms
image 7/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/BMW with license plate_35.jpeg: 480x640 1 car_plate, 189.2ms
image 8/56 /content/drive/MyDrive/ML_Ass10/datasets/pred/images/Chevrolet Cor

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'car_plate'}
 orig_img: array([[[254, 236, 219],
         [254, 236, 219],
         [254, 236, 219],
         ...,
         [ 55,  85, 102],
         [ 53,  82,  97],
         [ 51,  78,  92]],
 
        [[254, 236, 219],
         [254, 236, 219],
         [254, 236, 219],
         ...,
         [ 46,  74,  85],
         [ 41,  67,  79],
         [ 39,  63,  75]],
 
        [[254, 236, 219],
         [254, 236, 219],
         [254, 236, 219],
         ...,
         [ 38,  63,  73],
         [ 37,  62,  72],
         [ 42,  66,  78]],
 
        ...,
 
        [[ 80, 104, 124],
         [ 81, 106, 126],
         [ 81, 108, 128],
         ...,
         [126, 132, 131],
         [126, 132, 131],
         [126, 132, 131]],
 
        [[ 86, 108, 126],
         [ 84, 106, 124],
         [ 82, 106, 124],
         ...,
         [125, 131, 130],


---

# Appendix: Pipeline

```py
# Define functions for data processing
!pip install ultralytics

import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
import yaml
from ultralytics import YOLO

# region Arguements

ROOT_DIR = Path().resolve()  # Root path

LABEL_IMAGE_PATH = Path(ROOT_DIR, "data_label_image")
DATASET_PATH = Path(ROOT_DIR, "datasets")
TRAIN_DS_PATH = Path(DATASET_PATH, "train")
VAL_DS_PATH = Path(DATASET_PATH, "val")
PRED_DS_PATH = Path(DATASET_PATH, "pred")

TEST_SIZE = 0.15  # % of test size
VAL_SIZE = 0.15  # % of test size

YAML_PATH = "datasets/config.yaml"
BASE_MODEL = "./model/base/yolov8n.pt"

LABEL_NAME = "car_plate"

# endregion

# region Supporting Functions


def create_datasets_dir(dir):
    '''Create train and test directories'''

    if os.path.isdir(dir):
        shutil.rmtree(dir)

    Path(dir).mkdir(parents=True, exist_ok=True)

    train_path = Path(dir, "train")
    train_image_path = Path(train_path, "images")
    train_labels_path = Path(train_path, "labels")

    Path(train_path).mkdir(parents=True, exist_ok=True)
    Path(train_image_path).mkdir(parents=True, exist_ok=True)
    Path(train_labels_path).mkdir(parents=True, exist_ok=True)

    test_path = Path(dir, "test")
    test_image_path = Path(test_path, "images")
    test_labels_path = Path(test_path, "labels")

    Path(test_path).mkdir(parents=True, exist_ok=True)
    Path(test_image_path).mkdir(parents=True, exist_ok=True)
    Path(test_labels_path).mkdir(parents=True, exist_ok=True)

    test_path = Path(dir, "val")
    test_image_path = Path(test_path, "images")
    test_labels_path = Path(test_path, "labels")

    Path(test_path).mkdir(parents=True, exist_ok=True)
    Path(test_image_path).mkdir(parents=True, exist_ok=True)
    Path(test_labels_path).mkdir(parents=True, exist_ok=True)


def copy_files(source_fn_list, source_path,  target_path):
    '''Copy selected file list to destination'''

    image_path = Path(target_path, "images")
    labels_path = Path(target_path, "labels")
    for fn in source_fn_list:
        # if the csv file exists
        if Path(source_path, fn+".txt").is_file() and Path(source_path, fn+".jpeg").is_file():
            shutil.copy(Path(source_path, fn+".jpeg"), image_path)
            shutil.copy(Path(source_path, fn+".txt"), labels_path)


def report_info(target_path):
    '''report files in path'''
    report = {
        "images": {"path": Path(target_path, "images"), "count": 0},
        "labels": {"path": Path(target_path, "labels"), "count": 0},
    }

    for file in os.listdir(Path(target_path, "labels")):
        if file.endswith(".txt"):
            report['labels']["count"] += 1

    for file in os.listdir(Path(target_path, "images")):
        if file.endswith(".jpeg"):
            report["images"]["count"] += 1

    return report


def clear_runr_dir(dir):
    # Clean left over runs dir
    if os.path.isdir(dir):
        shutil.rmtree("./runs")

# endregion

# region Main()


def main():

    # establish datasets directory
    create_datasets_dir(DATASET_PATH)

    # get unique list of file
    fn_list = os.listdir(LABEL_IMAGE_PATH)
    fn_unique_list = list(set(["".join(fn.split(".")[:-1]) for fn in fn_list]))

    # split train, val, and pred dataset
    fn_train, fn_val_pred = train_test_split(
        fn_unique_list, test_size=VAL_SIZE+PRED_SIZE, random_state=42)    # get train set, with test_size = val+pred

    fn_val, fn_pred = train_test_split(
        fn_val_pred, test_size=PRED_SIZE/(PRED_SIZE+VAL_SIZE), random_state=42)    # get val and pred set, with test_size = pred/val+pred

    # # clone files to destination
    copy_files(fn_train, LABEL_IMAGE_PATH,  TRAIN_DS_PATH)
    copy_files(fn_val, LABEL_IMAGE_PATH,  VAL_DS_PATH)
    copy_files(fn_pred, LABEL_IMAGE_PATH,  PRED_DS_PATH)

    # train_report
    train_report = report_info(TRAIN_DS_PATH)
    print(
        f"\nReport: Train Dataset\nImages Path: {train_report['images']['path']}\nImages Count: {train_report['images']['count']}\nLabel Path: {train_report['labels']['path']}\nLabels Count: {train_report['labels']['count']}")

    # val_report
    val_report = report_info(VAL_DS_PATH)
    print(
        f"\nReport: Val Dataset\nImages Path: {val_report['images']['path']}\nImages Count: {val_report['images']['count']}\nLabel Path: {val_report['labels']['path']}\nLabels Count: {val_report['labels']['count']}")

    # pred_report
    pred_report = report_info(PRED_DS_PATH)
    print(
        f"\nReport: Pred Dataset\nImages Path: {pred_report['images']['path']}\nImages Count: {pred_report['images']['count']}\nLabel Path: {pred_report['labels']['path']}\nLabels Count: {pred_report['labels']['count']}")

    clear_runr_dir("./runs")

    # create yaml file
    data = dict(
        train=Path(TRAIN_DS_PATH, "images").absolute().as_posix(),
        val=Path(TRAIN_DS_PATH, "images").absolute().as_posix(),
        names={0: LABEL_NAME}
    )

    with open(YAML_PATH, 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)

    # Define model object from default v8n model
    car_plate_model = YOLO(BASE_MODEL)
    car_plate_model.train(data=YAML_PATH)    # Train Yolo model

# endregion

if __name__ == "__main__":
    main()
```